<center><img src=img/MScAI_brand.png width=70%></center>

# Comprehensions and Generators

Python aims to allow quite *lego-like* programming - small objects, flexibly combinable. Comprehensions and generators are two nice tools for this.

Comprehensions
---

A *comprehension* is a very handy syntax for transforming and slicing-and-dicing lists, sets and dicts to make new ones.

List comprehensions
---


The list comprehension syntax is like an inverted for-loop inside square brackets:

In [1]:
# make a list which we will use in following examples.
L = [0, 1, 2, 3, 4]

In [2]:
M = [x**2 for x in L]
M

[0, 1, 4, 9, 16]

Here, the `x**2` is an expression which will be evaluated for each value of `x`, and the results will be collected to make the new list.

In [3]:
# now M is a list
type(M)

list

In [4]:
print(M)

[0, 1, 4, 9, 16]


The above code is exactly equivalent to:

In [5]:
M = []
for x in L:
    M.append(x**2)

However, the list comprehension is shorter, faster, and (subjectively) more readable.

The list comprehension syntax is intended to parallel the "set
builder" notation which is familiar in mathematics, eg $s = \{x^2 \; | \; x = 0, ..., 4\}$

Filtering
---

We can filter out items using if-statements inside the comprehension:

In [6]:
M = [x**2 for x in L if x % 2 == 0]
print(M) # we get only squares of even numbers

[0, 4, 16]


The above code is exactly equivalent to:


In [7]:
M = []
for x in L:
    if x % 2 == 0:
        M.append(x**2)

You can have double for-loops, or even more -- but it becomes less
readable than the equivalent for-loop, and should be used sparingly.

In [8]:
N = [[1, 2, 3], [4, 5, 6]] # nested lists
P = [x**2 for M in N for x in M] 

Note that using `for x in M for M in N` instead (ie putting the `for` parts the other way around) will crash. To see why, try writing out the equivalent double for-loop. Which has to go first, the `for M in N` or the `for x in M`?

Set and dict comprehensions
---

There are also *set comprehensions* and *dict comprehensions* that
do what you expect. The set syntax is `{}` instead of `[]`:

In [9]:
s = {x**2 for x in L}
type(s)
print(s)

{0, 1, 4, 9, 16}


The dict syntax is again `{}`, but with `key: value` pairs inside it instead of single items:

In [10]:
d = {x: x**2 for x in L}
print(d)

{0: 0, 1: 1, 2: 4, 3: 9, 4: 16}


Reminder: you can loop over the keys of a dict with `for`:

In [11]:
for k in d:
    print(d[k])

0
1
4
9
16


Exercises
---

Given a dictionary `d` as follows:

In [12]:
d = {'a': 17, 'b': 20, 'c': 25}


1. Create the inverted dictionary `{17: 'a', 20: 'b', 25: 'c'}`.

2. Create a set of the keys `{'a', 'b', 'c'}`.

3. Create a list of the values `[17, 20, 25]`.

4. Create a list of the keys whose values are odd `['a', 'c']`.

Solution
---

In [13]:
{d[k]: k for k in d}

{17: 'a', 20: 'b', 25: 'c'}

In [14]:
{k for k in d}

{'a', 'b', 'c'}

In [15]:
[d[k] for k in d]

[17, 20, 25]

In [16]:
[k for k in d if d[k] % 2 == 1]

['a', 'c']

Generators
---

A generator is like a function -- but instead of returning a
single value, it *yields* one value at a time. 

* `yield`
* use inside for-loops
* save memory
* "lazy" versus "eager"

See also http://stackoverflow.com/questions/231767/what-does-the-yield-keyword-do-in-python

After the caller has "used" the `yield`ed value -- usually,
inside a `for` loop or comprehension or similar -- 
control comes back into the generator, and it can then proceed to yield
another value, and so on.

Generators are very useful in situations where we would normally have
to create a huge (potentially infinite) list as our return value,
which would use up all our RAM. A generator allows us to create and
yield just one item from the huge list at a time, so the huge list is
never actually formed.

A generator is called "lazy", because it doesn't do all its work
immediately when you call it. It does just enough for now, then
stops, and can resume later. A normal function is "eager", the opposite of lazy.


In [17]:
import random
# given a word, rearrange all but the first and last letter, eg 
# definitely -> dileefnity. 
def g(word):
    yield word[0]
    if len(word) > 1:
        rest = list(word[1:-1])
        random.shuffle(rest)
        for r in rest:
            yield r
        yield word[-1]

Calling a generator doesn't actually give us a yielded value: it just gives the generator itself, a piece of code ready to run. It is *iterable*, and now we have to iterate over it in some way.

In [18]:
g("definitely")

<generator object g at 0x10cdb1408>

In [19]:
for x in g("definitely"): print(x, end="")

dfenliiety

There's no law that says the consumer has to *use* all the items yielded by the generator:

In [20]:
i = 0
for x in g("arguably"):
    print(x, end="")
    if i > 3:
        break # later items will never be created
    i += 1

aalbu

This fact allows generators to be infinite, e.g. using 
```python
while True:
    yield something
```

In the `itertools` module there are several examples where this is useful.

Generator comprehensions
---

Apart from using `yield`, another way to create a generator is as a "generator comprehension". It's just like a list comprehension we saw before, but lazy. This time, the syntax is `()`:

In [21]:
gc = (x**2 for x in L)

Generator comprehensions are *used* by something that consumes a sequence -- like a `for` loop, or another comprehension:

In [22]:
for item in gc:
    print(item)

0
1
4
9
16


Notice that after we run through the generator, it is "used up", and now empty:

In [23]:
for item in gc:
    print(item)

We have to make the generator again if we need it:

In [24]:
gc = (x**2 for x in L)
M = [item+3 for item in gc]
print(M)

[3, 4, 7, 12, 19]


So a nice tactic is to write a function to create the generator, if we're going to need it more than once:

In [25]:
def make_gc(L):
    # this returns a generator. make_gc itself is NOT a generator
    return (x**2 for x in L) 
M = [item + 3 for item in make_gc(L)]
M

[3, 4, 7, 12, 19]

**Example** (adapted from Prof Michael Madden): Use a generator comprehension to generate Pythagorean triples (i.e. integers $(x, y, z)$ such that $x^2 + y^2 = z^2$), and $x, y, z \in [1, 30]$


In [26]:
triples = ((x,y,z) 
           for x in range(1,30)
           for y in range(x,30) 
           for z in range(y,30) 
           if x**2 + y**2 == z**2)
# notice we *canonicalise* on the ordering x <= y <= z to avoid duplicates
print('Pythagorean triples:')
for x, y, z in triples:
    print(x, y, z)

Pythagorean triples:
3 4 5
5 12 13
6 8 10
7 24 25
8 15 17
9 12 15
10 24 26
12 16 20
15 20 25
20 21 29


**Exercises**


* Write a generator comprehension to generate all cuboids with integer sides each < 250. 

* Put that inside a function as described above since we'll be using it several times.

* Use `sum` to count the number of elements in the generator.

* Use a for-loop over the generator to find the cuboids whose sides sum to 250.

* Of those, find the cuboid of *maximum volume*.

In [27]:
top = 250
def cuboids(): 
    return ( # leave a bracket open: multi-line expression...
        (x, y, z)
        for x in range(top) 
        for y in range(top) 
        for z in range(top) 
    ) # ... close the bracket
sum(1 for r in cuboids())

15625000

In [28]:
for c in cuboids():
    if c[0] + c[1] + c[2] == top:
        print(c)

(0, 1, 249)
(0, 2, 248)
(0, 3, 247)
(0, 4, 246)
(0, 5, 245)
(0, 6, 244)
(0, 7, 243)
(0, 8, 242)
(0, 9, 241)
(0, 10, 240)
(0, 11, 239)
(0, 12, 238)
(0, 13, 237)
(0, 14, 236)
(0, 15, 235)
(0, 16, 234)
(0, 17, 233)
(0, 18, 232)
(0, 19, 231)
(0, 20, 230)
(0, 21, 229)
(0, 22, 228)
(0, 23, 227)
(0, 24, 226)
(0, 25, 225)
(0, 26, 224)
(0, 27, 223)
(0, 28, 222)
(0, 29, 221)
(0, 30, 220)
(0, 31, 219)
(0, 32, 218)
(0, 33, 217)
(0, 34, 216)
(0, 35, 215)
(0, 36, 214)
(0, 37, 213)
(0, 38, 212)
(0, 39, 211)
(0, 40, 210)
(0, 41, 209)
(0, 42, 208)
(0, 43, 207)
(0, 44, 206)
(0, 45, 205)
(0, 46, 204)
(0, 47, 203)
(0, 48, 202)
(0, 49, 201)
(0, 50, 200)
(0, 51, 199)
(0, 52, 198)
(0, 53, 197)
(0, 54, 196)
(0, 55, 195)
(0, 56, 194)
(0, 57, 193)
(0, 58, 192)
(0, 59, 191)
(0, 60, 190)
(0, 61, 189)
(0, 62, 188)
(0, 63, 187)
(0, 64, 186)
(0, 65, 185)
(0, 66, 184)
(0, 67, 183)
(0, 68, 182)
(0, 69, 181)
(0, 70, 180)
(0, 71, 179)
(0, 72, 178)
(0, 73, 177)
(0, 74, 176)
(0, 75, 175)
(0, 76, 174)
(0, 77, 173)
(0, 78, 

(5, 47, 198)
(5, 48, 197)
(5, 49, 196)
(5, 50, 195)
(5, 51, 194)
(5, 52, 193)
(5, 53, 192)
(5, 54, 191)
(5, 55, 190)
(5, 56, 189)
(5, 57, 188)
(5, 58, 187)
(5, 59, 186)
(5, 60, 185)
(5, 61, 184)
(5, 62, 183)
(5, 63, 182)
(5, 64, 181)
(5, 65, 180)
(5, 66, 179)
(5, 67, 178)
(5, 68, 177)
(5, 69, 176)
(5, 70, 175)
(5, 71, 174)
(5, 72, 173)
(5, 73, 172)
(5, 74, 171)
(5, 75, 170)
(5, 76, 169)
(5, 77, 168)
(5, 78, 167)
(5, 79, 166)
(5, 80, 165)
(5, 81, 164)
(5, 82, 163)
(5, 83, 162)
(5, 84, 161)
(5, 85, 160)
(5, 86, 159)
(5, 87, 158)
(5, 88, 157)
(5, 89, 156)
(5, 90, 155)
(5, 91, 154)
(5, 92, 153)
(5, 93, 152)
(5, 94, 151)
(5, 95, 150)
(5, 96, 149)
(5, 97, 148)
(5, 98, 147)
(5, 99, 146)
(5, 100, 145)
(5, 101, 144)
(5, 102, 143)
(5, 103, 142)
(5, 104, 141)
(5, 105, 140)
(5, 106, 139)
(5, 107, 138)
(5, 108, 137)
(5, 109, 136)
(5, 110, 135)
(5, 111, 134)
(5, 112, 133)
(5, 113, 132)
(5, 114, 131)
(5, 115, 130)
(5, 116, 129)
(5, 117, 128)
(5, 118, 127)
(5, 119, 126)
(5, 120, 125)
(5, 121, 124)
(5,

(10, 173, 67)
(10, 174, 66)
(10, 175, 65)
(10, 176, 64)
(10, 177, 63)
(10, 178, 62)
(10, 179, 61)
(10, 180, 60)
(10, 181, 59)
(10, 182, 58)
(10, 183, 57)
(10, 184, 56)
(10, 185, 55)
(10, 186, 54)
(10, 187, 53)
(10, 188, 52)
(10, 189, 51)
(10, 190, 50)
(10, 191, 49)
(10, 192, 48)
(10, 193, 47)
(10, 194, 46)
(10, 195, 45)
(10, 196, 44)
(10, 197, 43)
(10, 198, 42)
(10, 199, 41)
(10, 200, 40)
(10, 201, 39)
(10, 202, 38)
(10, 203, 37)
(10, 204, 36)
(10, 205, 35)
(10, 206, 34)
(10, 207, 33)
(10, 208, 32)
(10, 209, 31)
(10, 210, 30)
(10, 211, 29)
(10, 212, 28)
(10, 213, 27)
(10, 214, 26)
(10, 215, 25)
(10, 216, 24)
(10, 217, 23)
(10, 218, 22)
(10, 219, 21)
(10, 220, 20)
(10, 221, 19)
(10, 222, 18)
(10, 223, 17)
(10, 224, 16)
(10, 225, 15)
(10, 226, 14)
(10, 227, 13)
(10, 228, 12)
(10, 229, 11)
(10, 230, 10)
(10, 231, 9)
(10, 232, 8)
(10, 233, 7)
(10, 234, 6)
(10, 235, 5)
(10, 236, 4)
(10, 237, 3)
(10, 238, 2)
(10, 239, 1)
(10, 240, 0)
(11, 0, 239)
(11, 1, 238)
(11, 2, 237)
(11, 3, 236)
(11, 4

(16, 0, 234)
(16, 1, 233)
(16, 2, 232)
(16, 3, 231)
(16, 4, 230)
(16, 5, 229)
(16, 6, 228)
(16, 7, 227)
(16, 8, 226)
(16, 9, 225)
(16, 10, 224)
(16, 11, 223)
(16, 12, 222)
(16, 13, 221)
(16, 14, 220)
(16, 15, 219)
(16, 16, 218)
(16, 17, 217)
(16, 18, 216)
(16, 19, 215)
(16, 20, 214)
(16, 21, 213)
(16, 22, 212)
(16, 23, 211)
(16, 24, 210)
(16, 25, 209)
(16, 26, 208)
(16, 27, 207)
(16, 28, 206)
(16, 29, 205)
(16, 30, 204)
(16, 31, 203)
(16, 32, 202)
(16, 33, 201)
(16, 34, 200)
(16, 35, 199)
(16, 36, 198)
(16, 37, 197)
(16, 38, 196)
(16, 39, 195)
(16, 40, 194)
(16, 41, 193)
(16, 42, 192)
(16, 43, 191)
(16, 44, 190)
(16, 45, 189)
(16, 46, 188)
(16, 47, 187)
(16, 48, 186)
(16, 49, 185)
(16, 50, 184)
(16, 51, 183)
(16, 52, 182)
(16, 53, 181)
(16, 54, 180)
(16, 55, 179)
(16, 56, 178)
(16, 57, 177)
(16, 58, 176)
(16, 59, 175)
(16, 60, 174)
(16, 61, 173)
(16, 62, 172)
(16, 63, 171)
(16, 64, 170)
(16, 65, 169)
(16, 66, 168)
(16, 67, 167)
(16, 68, 166)
(16, 69, 165)
(16, 70, 164)
(16, 71, 163)
(1

(21, 228, 1)
(21, 229, 0)
(22, 0, 228)
(22, 1, 227)
(22, 2, 226)
(22, 3, 225)
(22, 4, 224)
(22, 5, 223)
(22, 6, 222)
(22, 7, 221)
(22, 8, 220)
(22, 9, 219)
(22, 10, 218)
(22, 11, 217)
(22, 12, 216)
(22, 13, 215)
(22, 14, 214)
(22, 15, 213)
(22, 16, 212)
(22, 17, 211)
(22, 18, 210)
(22, 19, 209)
(22, 20, 208)
(22, 21, 207)
(22, 22, 206)
(22, 23, 205)
(22, 24, 204)
(22, 25, 203)
(22, 26, 202)
(22, 27, 201)
(22, 28, 200)
(22, 29, 199)
(22, 30, 198)
(22, 31, 197)
(22, 32, 196)
(22, 33, 195)
(22, 34, 194)
(22, 35, 193)
(22, 36, 192)
(22, 37, 191)
(22, 38, 190)
(22, 39, 189)
(22, 40, 188)
(22, 41, 187)
(22, 42, 186)
(22, 43, 185)
(22, 44, 184)
(22, 45, 183)
(22, 46, 182)
(22, 47, 181)
(22, 48, 180)
(22, 49, 179)
(22, 50, 178)
(22, 51, 177)
(22, 52, 176)
(22, 53, 175)
(22, 54, 174)
(22, 55, 173)
(22, 56, 172)
(22, 57, 171)
(22, 58, 170)
(22, 59, 169)
(22, 60, 168)
(22, 61, 167)
(22, 62, 166)
(22, 63, 165)
(22, 64, 164)
(22, 65, 163)
(22, 66, 162)
(22, 67, 161)
(22, 68, 160)
(22, 69, 159)
(22,

...

(169, 17, 64)
(169, 18, 63)
(169, 19, 62)
(169, 20, 61)
(169, 21, 60)
(169, 22, 59)
(169, 23, 58)
(169, 24, 57)
(169, 25, 56)
(169, 26, 55)
(169, 27, 54)
(169, 28, 53)
(169, 29, 52)
(169, 30, 51)
(169, 31, 50)
(169, 32, 49)
(169, 33, 48)
(169, 34, 47)
(169, 35, 46)
(169, 36, 45)
(169, 37, 44)
(169, 38, 43)
(169, 39, 42)
(169, 40, 41)
(169, 41, 40)
(169, 42, 39)
(169, 43, 38)
(169, 44, 37)
(169, 45, 36)
(169, 46, 35)
(169, 47, 34)
(169, 48, 33)
(169, 49, 32)
(169, 50, 31)
(169, 51, 30)
(169, 52, 29)
(169, 53, 28)
(169, 54, 27)
(169, 55, 26)
(169, 56, 25)
(169, 57, 24)
(169, 58, 23)
(169, 59, 22)
(169, 60, 21)
(169, 61, 20)
(169, 62, 19)
(169, 63, 18)
(169, 64, 17)
(169, 65, 16)
(169, 66, 15)
(169, 67, 14)
(169, 68, 13)
(169, 69, 12)
(169, 70, 11)
(169, 71, 10)
(169, 72, 9)
(169, 73, 8)
(169, 74, 7)
(169, 75, 6)
(169, 76, 5)
(169, 77, 4)
(169, 78, 3)
(169, 79, 2)
(169, 80, 1)
(169, 81, 0)
(170, 0, 80)
(170, 1, 79)
(170, 2, 78)
(170, 3, 77)
(170, 4, 76)
(170, 5, 75)
(170, 6, 74)
(170, 7, 

(177, 17, 56)
(177, 18, 55)
(177, 19, 54)
(177, 20, 53)
(177, 21, 52)
(177, 22, 51)
(177, 23, 50)
(177, 24, 49)
(177, 25, 48)
(177, 26, 47)
(177, 27, 46)
(177, 28, 45)
(177, 29, 44)
(177, 30, 43)
(177, 31, 42)
(177, 32, 41)
(177, 33, 40)
(177, 34, 39)
(177, 35, 38)
(177, 36, 37)
(177, 37, 36)
(177, 38, 35)
(177, 39, 34)
(177, 40, 33)
(177, 41, 32)
(177, 42, 31)
(177, 43, 30)
(177, 44, 29)
(177, 45, 28)
(177, 46, 27)
(177, 47, 26)
(177, 48, 25)
(177, 49, 24)
(177, 50, 23)
(177, 51, 22)
(177, 52, 21)
(177, 53, 20)
(177, 54, 19)
(177, 55, 18)
(177, 56, 17)
(177, 57, 16)
(177, 58, 15)
(177, 59, 14)
(177, 60, 13)
(177, 61, 12)
(177, 62, 11)
(177, 63, 10)
(177, 64, 9)
(177, 65, 8)
(177, 66, 7)
(177, 67, 6)
(177, 68, 5)
(177, 69, 4)
(177, 70, 3)
(177, 71, 2)
(177, 72, 1)
(177, 73, 0)
(178, 0, 72)
(178, 1, 71)
(178, 2, 70)
(178, 3, 69)
(178, 4, 68)
(178, 5, 67)
(178, 6, 66)
(178, 7, 65)
(178, 8, 64)
(178, 9, 63)
(178, 10, 62)
(178, 11, 61)
(178, 12, 60)
(178, 13, 59)
(178, 14, 58)
(178, 15, 57

(192, 0, 58)
(192, 1, 57)
(192, 2, 56)
(192, 3, 55)
(192, 4, 54)
(192, 5, 53)
(192, 6, 52)
(192, 7, 51)
(192, 8, 50)
(192, 9, 49)
(192, 10, 48)
(192, 11, 47)
(192, 12, 46)
(192, 13, 45)
(192, 14, 44)
(192, 15, 43)
(192, 16, 42)
(192, 17, 41)
(192, 18, 40)
(192, 19, 39)
(192, 20, 38)
(192, 21, 37)
(192, 22, 36)
(192, 23, 35)
(192, 24, 34)
(192, 25, 33)
(192, 26, 32)
(192, 27, 31)
(192, 28, 30)
(192, 29, 29)
(192, 30, 28)
(192, 31, 27)
(192, 32, 26)
(192, 33, 25)
(192, 34, 24)
(192, 35, 23)
(192, 36, 22)
(192, 37, 21)
(192, 38, 20)
(192, 39, 19)
(192, 40, 18)
(192, 41, 17)
(192, 42, 16)
(192, 43, 15)
(192, 44, 14)
(192, 45, 13)
(192, 46, 12)
(192, 47, 11)
(192, 48, 10)
(192, 49, 9)
(192, 50, 8)
(192, 51, 7)
(192, 52, 6)
(192, 53, 5)
(192, 54, 4)
(192, 55, 3)
(192, 56, 2)
(192, 57, 1)
(192, 58, 0)
(193, 0, 57)
(193, 1, 56)
(193, 2, 55)
(193, 3, 54)
(193, 4, 53)
(193, 5, 52)
(193, 6, 51)
(193, 7, 50)
(193, 8, 49)
(193, 9, 48)
(193, 10, 47)
(193, 11, 46)
(193, 12, 45)
(193, 13, 44)
(193, 14

(211, 0, 39)
(211, 1, 38)
(211, 2, 37)
(211, 3, 36)
(211, 4, 35)
(211, 5, 34)
(211, 6, 33)
(211, 7, 32)
(211, 8, 31)
(211, 9, 30)
(211, 10, 29)
(211, 11, 28)
(211, 12, 27)
(211, 13, 26)
(211, 14, 25)
(211, 15, 24)
(211, 16, 23)
(211, 17, 22)
(211, 18, 21)
(211, 19, 20)
(211, 20, 19)
(211, 21, 18)
(211, 22, 17)
(211, 23, 16)
(211, 24, 15)
(211, 25, 14)
(211, 26, 13)
(211, 27, 12)
(211, 28, 11)
(211, 29, 10)
(211, 30, 9)
(211, 31, 8)
(211, 32, 7)
(211, 33, 6)
(211, 34, 5)
(211, 35, 4)
(211, 36, 3)
(211, 37, 2)
(211, 38, 1)
(211, 39, 0)
(212, 0, 38)
(212, 1, 37)
(212, 2, 36)
(212, 3, 35)
(212, 4, 34)
(212, 5, 33)
(212, 6, 32)
(212, 7, 31)
(212, 8, 30)
(212, 9, 29)
(212, 10, 28)
(212, 11, 27)
(212, 12, 26)
(212, 13, 25)
(212, 14, 24)
(212, 15, 23)
(212, 16, 22)
(212, 17, 21)
(212, 18, 20)
(212, 19, 19)
(212, 20, 18)
(212, 21, 17)
(212, 22, 16)
(212, 23, 15)
(212, 24, 14)
(212, 25, 13)
(212, 26, 12)
(212, 27, 11)
(212, 28, 10)
(212, 29, 9)
(212, 30, 8)
(212, 31, 7)
(212, 32, 6)
(212, 33, 5)

(243, 0, 7)
(243, 1, 6)
(243, 2, 5)
(243, 3, 4)
(243, 4, 3)
(243, 5, 2)
(243, 6, 1)
(243, 7, 0)
(244, 0, 6)
(244, 1, 5)
(244, 2, 4)
(244, 3, 3)
(244, 4, 2)
(244, 5, 1)
(244, 6, 0)
(245, 0, 5)
(245, 1, 4)
(245, 2, 3)
(245, 3, 2)
(245, 4, 1)
(245, 5, 0)
(246, 0, 4)
(246, 1, 3)
(246, 2, 2)
(246, 3, 1)
(246, 4, 0)
(247, 0, 3)
(247, 1, 2)
(247, 2, 1)
(247, 3, 0)
(248, 0, 2)
(248, 1, 1)
(248, 2, 0)
(249, 0, 1)
(249, 1, 0)


In [29]:
def volume(c): return c[0] * c[1] * c[2]
max((c for c in cuboids() 
     if c[0] + c[1] + c[2] == top), # another generator comprehension 
    key=volume)

(83, 83, 84)